In [26]:
import getpass
import os
import requests
import json
import time

from langchain.chat_models.gigachat import GigaChat
from langchain.memory import (
    ConversationBufferMemory,
    ConversationBufferWindowMemory,
    ConversationSummaryMemory,
    VectorStoreRetrieverMemory,
    ConversationKGMemory
)
from langchain.chains import ConversationChain
from langchain_community.embeddings import GigaChatEmbeddings
from langchain_core.prompts.prompt import PromptTemplate

from langchain.schema import HumanMessage, SystemMessage, AIMessage
from langchain.tools import tool

import faiss
from langchain_community.docstore import InMemoryDocstore
from langchain_community.vectorstores import FAISS

from pydantic import BaseModel

In [2]:
from dotenv import load_dotenv
# Load environment variables
load_dotenv('.env')

True

# 1. Memory

Define your own class implementing a simple LLM-based chatbot. You need to use at least three memory types (langchain.memory), which are set as one argument in the ```init``` definition. If the memory type has any parameters, you also need to define them as arguments in the ```init``` definition. You also need to define a ```run``` method implementing the main conversation loop, and a ```print_memory``` method to print out what exactly the memory consists of.

In [3]:
class SimpleChatBot:
    def __init__(self, llm, memory_type, **memory_params):
        self.llm = llm
        match memory_type:
            case 'buffer':
                self.memory = ConversationBufferMemory(**memory_params)
            case 'summary':
                self.memory = ConversationSummaryMemory(llm=llm, **memory_params)
            case 'window':
                self.memory = ConversationBufferWindowMemory(llm=llm, **memory_params)
            case _:
                raise ValueError(f"Memory type '{memory_type}' is not supported.")

    def _respond(self, user_input):
        context = self.memory.load_memory_variables({})
        response = self.llm.predict(
            text=user_input,
            context=context
        )
        self.memory.save_context({"input": user_input}, {"output": response})
        return response

    
    def print_memory(self):
        print(self.memory.load_memory_variables({}))


    def run(self):
        print("This is a simple chat bot:")
        city = str(input("Город: "))
        user_input = PromptTemplate(template='Расскажи в одном предложении про город {city}',
                                    input_variables=["city"],
                                    input_types={"city": str})
        response = self._respond(user_input.format(city=city))
        print(response)


Now let's check how it works with each type of memory

In [4]:
giga_key = os.environ.get("SB_AUTH_DATA")
giga = GigaChat(credentials=giga_key, model="GigaChat", timeout=30, verify_ssl_certs=False)
chat = SimpleChatBot(giga, 'summary')
chat.run()
chat.print_memory()

This is a simple chat bot:


/var/folders/t3/409cw_p51s3_b296bytqrv3x8ch15l/T/ipykernel_17733/2987679950.py:16: LangChainDeprecationWarning: The method `BaseChatModel.predict` was deprecated in langchain-core 0.1.7 and will be removed in 1.0. Use invoke instead.
  response = self.llm.predict(


Москва — столица России, крупнейший город страны и один из самых крупных городов мира.
{'history': 'AI кратко описывает Москву как столицу России и один из крупнейших городов мира.'}


In [5]:
giga_key = os.environ.get("SB_AUTH_DATA")
giga = GigaChat(credentials=giga_key, model="GigaChat", timeout=30, verify_ssl_certs=False)
chat = SimpleChatBot(giga, 'buffer')
chat.run()
chat.print_memory()

This is a simple chat bot:
Санкт-Петербург — это второй по величине город в России, известный своей богатой историей, архитектурой и ролью в культурной жизни страны.
{'history': 'Human: Расскажи в одном предложении про город Санкт-Петербург\nAI: Санкт-Петербург — это второй по величине город в России, известный своей богатой историей, архитектурой и ролью в культурной жизни страны.'}


In [6]:
giga_key = os.environ.get("SB_AUTH_DATA")
giga = GigaChat(credentials=giga_key, model="GigaChat", timeout=30, verify_ssl_certs=False)
chat = SimpleChatBot(giga, 'window', k=2)
chat.run()
chat.print_memory()

This is a simple chat bot:
Санкт-Петербург — это второй по величине город в России, известный своей богатой историей, архитектурой и ролью в культурной жизни страны.
{'history': 'Human: Расскажи в одном предложении про город Санкт-Петербург\nAI: Санкт-Петербург — это второй по величине город в России, известный своей богатой историей, архитектурой и ролью в культурной жизни страны.'}


In [7]:
chat.run()
chat.print_memory()

This is a simple chat bot:
Санкт-Петербург — это второй по величине город в России, известный своей богатой историей, архитектурой и ролью в культурной жизни страны.
{'history': 'Human: Расскажи в одном предложении про город Санкт-Петербург\nAI: Санкт-Петербург — это второй по величине город в России, известный своей богатой историей, архитектурой и ролью в культурной жизни страны.\nHuman: Расскажи в одном предложении про город Санкт-Петербург\nAI: Санкт-Петербург — это второй по величине город в России, известный своей богатой историей, архитектурой и ролью в культурной жизни страны.'}


In [8]:
chat.run()
chat.print_memory()

This is a simple chat bot:
Санкт-Петербург — это второй по величине город в России, известный своей богатой историей, архитектурой и ролью в культурной жизни страны.
{'history': 'Human: Расскажи в одном предложении про город Санкт-Петербург\nAI: Санкт-Петербург — это второй по величине город в России, известный своей богатой историей, архитектурой и ролью в культурной жизни страны.\nHuman: Расскажи в одном предложении про город Санкт-Петербург\nAI: Санкт-Петербург — это второй по величине город в России, известный своей богатой историей, архитектурой и ролью в культурной жизни страны.'}


Please make a short report abount differences between used memory types

Report:

- Buffer:
    - Сохраняет весь разговор в линейной последовательности.
    -   Преимущество: Полный доступ к предыдущим сообщениям.
    -   Недостаток: Память разрастается с увеличением диалога, что может снизить производительность.

- Summary:
    - Обобщает предыдущие сообщения, создавая краткий обзор беседы.
    - Преимущество: Экономит память и улучшает контекстное восприятие за счет сокращенных данных.
    - Недостаток: Возможны потери деталей, которые могут оказаться важными.

- Window:
    - Хранит только определенное количество последних сообщений (окно).
    - Преимущество: Легкая и быстрая работа за счет ограничения объема данных.
    - Недостаток: Нет доступа к старым частям разговора, которые выходят за пределы окна.


# 2. Using tools and agents

## 2.1 Using tools and API

Create your own tool based on the langchain.tools library to interact with a public OpenWeather API. This tool will receive data from the API and return it as a readable result for the user.


OpenWeather API URL: https://api.openweathermap.org/data/2.5/weather?q={city}&appid={openweather_key}&units=metric 

[How to get OpenWeather API key](https://docs.google.com/document/d/1vbi8QKqMZqZoCReIzpmEB_2mHsrbmXPlyGngE3jeDDw/edit)


In [9]:
from langchain_core.pydantic_v1 import BaseModel, Field
from typing import List, Optional

from langchain.tools import tool
from langchain.agents import AgentExecutor, create_gigachat_functions_agent
from langchain.chat_models.gigachat import GigaChat


openweather_key = os.getenv("OPENWEATHER_API_KEY")

class WeatherResult(BaseModel):
    city: str = Field(description="Название города")
    temperature: float = Field(description="Температура в указанном городе в градусах Цельсия")
    feels_like: float = Field(description="Температура, как она ощущается в градусах Цельсия")
    weather: str = Field(description="Краткое описание текущей погоды в городе")
    wind: float = Field(description="Скорость ветра в метрах в секунду")
    message: str = Field(description="Сообщение о статусе операции")

few_shot_examples = [
    {
        "request": "Какая погода сейчас в Ставрополе?",
        "params": {"city": "Ставрополь"},
    },
    {
        "request": "Организуй поездку из Москвы в Санкт-Петербург на 10 дней с 15.11.2024 - отель, самолет, погода",
        "params": {"city": "Санкт-Петербург"},
    },
    {
        "request": "Организуй поездку в Санкт-Петербурге на 10 дней с 15.11.2024 - отель, самолет, погода",
        "params": {"city": "Санкт-Петербург"},
    }
]

@tool(few_shot_examples=few_shot_examples)
def get_weather(
    city: str = Field(description="Название города для получения погоды")
) -> WeatherResult:
    """
    Получаем текущие погодные данные для заданного города через OpenWeather API и возвращаем результат.
    """
    base_url = f"https://api.openweathermap.org/data/2.5/weather?q={city}&appid={openweather_key}&units=metric"
    try:
        response = requests.get(base_url)
        response.raise_for_status()
        data = response.json()
        
        city_name = data['name']
        temperature = data['main']['temp']
        feels_like = data['main']['feels_like']
        weather_description = data['weather'][0]['description']
        wind_speed = data['wind']['speed']
        
        return WeatherResult(
            city=city_name,
            temperature=temperature,
            feels_like=feels_like,
            weather=weather_description,
            wind=wind_speed,
            message="Successfull"
        )
    except requests.exceptions.RequestException as e:
        return WeatherResult(city=city, temperature=0, feels_like=0, weather="", wind=0, message=f"Ошибка при выполнении запроса: {str(e)}")
    except KeyError:
        return WeatherResult(city=city, temperature=0, feels_like=0, weather="", wind=0, message="Ошибка в данных ответа сервера.")


class OpenWeatherAPITool:
    def __init__(self, llm, agent_function):
        self.llm = llm
        self.agent_function = agent_function
        self.tools = [agent_function]
        self.agent = create_gigachat_functions_agent(llm, self.tools)
        self.agent_executor = AgentExecutor(
            agent=self.agent,
            tools=self.tools,
            verbose=True,
        )


    def run(self, user_input: str):
        return self.agent_executor.invoke({"input": user_input})

Let's check it

In [10]:
giga_key = os.environ.get("SB_AUTH_DATA")
giga_pro = GigaChat(credentials=giga_key, model="GigaChat", timeout=30, verify_ssl_certs=False)

openwheatertool = OpenWeatherAPITool(giga_pro, get_weather)
user_input = "Какая погода сейчас в Санкт-Петербурге?"
openwheatertool.run(user_input)



> Entering new AgentExecutor chain...

Invoking: `get_weather` with `{'city': 'Санкт-Петербург'}`


city='Saint Petersburg' temperature=8.08 feels_like=6.21 weather='clear sky' wind=3.0 message='Successfull'Сейчас в Санкт-Петербурге ясная погода с температурой около 8 градусов Цельсия.

> Finished chain.


{'input': 'Какая погода сейчас в Санкт-Петербурге?',
 'output': 'Сейчас в Санкт-Петербурге ясная погода с температурой около 8 градусов Цельсия.'}

## 2.2. Multi agents

Create a multi-agent system where each agent is responsible for a specific task in the travel planning process. For example, one agent is responsible for searching for flights, another for booking hotels, and a third for finding the weather at the destination.

Requirements:

- Use three or more GigaChat-based agents to interact with each other.
- The first agent is responsible for searching for flights (using ```get_url_booking_tickets``` function).
- The second agent is responsible for booking hotels (using ```get_url_booking_hotels``` function).
- The third agent collects weather information for the destination (using a real API, such as OpenWeather). You can use the function from the previous task (for simplify, here you can give a current weather, not a forecast for the specific date)

In [67]:
from langchain.tools import Tool
from langchain.agents import create_gigachat_functions_agent, AgentExecutor
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder, SystemMessagePromptTemplate, HumanMessagePromptTemplate
import requests
import json
from datetime import datetime, timedelta
from dateutil import parser

openweather_key = os.environ.get("OPENWEATHER_API_KEY")

def get_geoid(city: str) -> str:
    url_base = 'https://suggest-maps.yandex.ru/suggest-geo'
    params = {
        'search_type': 'tune',
        'v': '9',
        'results': 1,
        'lang': 'ry_RU',
        'callback': 'json',
        'part': city,
    }
    r = requests.get(url_base, params=params)
    if r.ok:
        r_text = r.text
        r_json = r_text[5:-1]
        res_json = json.loads(r_json)
        res = res_json['results'][0]['geoid']
    else:
        res = ''
    return str(res)


class HotelResult(BaseModel):
    city: str = Field(description="Город для бронирования отеля")
    checkin_date: str = Field(description="Дата заезда в формате yyyy-mm-dd")
    checkout_date: str = Field(description="Дата выезда в формате yyyy-mm-dd")
    url: str = Field(description="Ссылка на поиск отелей")
    message: str = Field(description="Сообщение о статусе операции")
    
few_shot_examples_hotel = [
    {
        "request": "Организуй поездку в Санкт-Петербурге на 10 дней с 15.11.2024 - отель, самолет, погода",
        "params": {"city": "Санкт-Петербург", "date_in_str": "2024-11-15", "date_out_str": "2024-11-25"}
    },
    {
        "request": "Нужен отель в Москве с 15.11.2024, на 3 дня",
        "params": {"city": "Москва", "date_in_str": "2024-11-15", "date_out_str": "2024-11-18"}
    },
    {
        "request": "Организуй поездку из Москвы в Санкт-Петербург на 10 дней с 15.11.2024 - отель, самолет, погода",
        "params": {"city": "Санкт-Петербург", "date_in_str": "2024-11-15", "date_out_str": "2024-11-25"}
    }
]

@tool(few_shot_examples=few_shot_examples_hotel)
def get_url_booking_hotels(date_in_str: str, date_out_str: str, city: str) -> HotelResult:
    """Возврат ссылки для бронирования отелей в заданных датах и городе."""
    try:
        date_in = parser.parse(date_in_str)
        date_out = parser.parse(date_out_str)
        geoid = get_geoid(city)
        url = 'https://travel.yandex.ru/hotels/search/?'
        params = {
            'adults': '2',
            'checkinDate': date_in.strftime('%Y-%m-%d'),
            'checkoutDate': date_out.strftime('%Y-%m-%d'),
            'childrenAges': '0',
            'geoId': geoid
        }
        for item in params:
            url += f'&{item}={params[item]}'

        return HotelResult(
            city=city,
            checkin_date=date_in_str,
            checkout_date=date_out_str,
            url=f"Booking link for hotels in {city}: {url}",
            message="Hotel booking search successful"
        )
    except Exception as e:
        return HotelResult(city=city, checkin_date="", checkout_date="", url="", message=f"Error: {str(e)}")


class FlightResult(BaseModel):
    city_from: str = Field(description="Город отправления")
    city_to: str = Field(description="Город назначения")
    departure_date: str = Field(description="Дата вылета в формате yyyy-mm-dd")
    return_date: str = Field(description="Дата обратного рейса в формате yyyy-mm-dd")
    url: str = Field(description="Ссылка на поиск билетов")
    message: str = Field(description="Сообщение о статусе операции")

few_shot_flight_examples = [
    {
        "request": "Организуй поездку в Санкт-Петербурге на 10 дней с 15.11.2024 - отель, самолет, погода",
        "params": {"city_from": "Санкт-Петербург", "city_to": "Санкт-Петербург", "date_in_str": "2024-11-15", "date_out_str": "2024-11-25"}
    },
    {
        "request": "Организуй поездку из Москвы в Санкт-Петербург на 10 дней с 15.11.2024 - отель, самолет, погода",
        "params": {"city_from": "Москва", "city_to": "Санкт-Петербург", "date_in_str": "2024-11-15", "date_out_str": "2024-11-25"}
    }
]

@tool(few_shot_examples=few_shot_flight_examples)
def get_url_booking_tikets(city_from: str, city_to: str, date_in_str: str, date_out_str: str) -> FlightResult:
    """Возврат ссылки для бронирования билетов в указанные даты и города."""
    try:
        date_in = parser.parse(date_in_str)
        date_out = parser.parse(date_out_str)
        fromid = get_geoid(city_from)
        toid = get_geoid(city_to)

        url = 'https://travel.yandex.ru/avia/search/result/?'
        params = {
            'adults_seats': '2',
            'fromId': f'c{fromid}',
            'klass': 'economy',
            'oneway': '2',
            'return_date': date_out.strftime('%Y-%m-%d'),
            'toId': f'c{toid}',
            'when': date_in.strftime('%Y-%m-%d'),
        }

        for item, value in params.items():
            url += f'&{item}={value}'

        return FlightResult(
            city_from=city_from,
            city_to=city_to,
            departure_date=date_in_str,
            return_date=date_out_str,
            url=f"Flight booking link from {city_from} to {city_to}: {url}",
            message="Flight booking search successful"
        )
    except Exception as e:
        return FlightResult(city_from=city_from, city_to=city_to, departure_date="", return_date="", url="", message=f"Error: {str(e)}")


class MultiAgent:
    def __init__(self, llm, agent_function_weather, agent_function_hotels, agent_function_tickets):
        self.llm = llm
        self.agent_weather = self._create_agent(tools=[agent_function_weather],
                                                system_prompt="You are a weather assistant, your task is to retrieve the weather of a given city using the get_weather tool.")
        self.agent_hotels = self._create_agent(tools=[agent_function_hotels],
                                               system_prompt="You are a hotel assistant. You are responsible for finding hotels using the get_url_booking_hotels tool.")
        self.agent_tickets = self._create_agent(tools=[agent_function_tickets],
                                                system_prompt="You are a flight assistant, your task is to search for flights and use the get_url_booking_tickets tool.")
        self.postprocessing_template = PromptTemplate.from_template("""
        У тебя есть информация по:
        - погоде: {weather}
        - отелям: {hotels}
        - билетам: {tickets}
        Приведи ее в человекочитаемый формат (списком, без \символа tab) и выведи. Не теряй ссылки и даты.
        """)

    def _create_agent(self, tools, system_prompt):
        prompt = ChatPromptTemplate.from_messages([
            SystemMessage(system_prompt),
            MessagesPlaceholder(variable_name="messages"),
            MessagesPlaceholder(variable_name="agent_scratchpad"),
        ])

        agent = create_gigachat_functions_agent(self.llm, tools, prompt)
        return AgentExecutor(agent=agent, tools=tools, verbose=True)

    def run(self, user_input: str):
        weather = self.agent_weather.invoke({"messages": [HumanMessage(content=f"Определи погоду в городе из запроса: {user_input}")]})
        hotels = self.agent_hotels.invoke({"messages": [HumanMessage(content=f"Найди билеты в отель в городе назначения из запроса: {user_input}")]})
        tickets = self.agent_tickets.invoke({"messages": [HumanMessage(content=f"Найди авиабилеты из города отправления в город назначения из запроса: {user_input}")]})
        info_dict = {
            "weather": weather["output"],
            "hotels": hotels["output"],
            "tickets": tickets["output"]
        }
        postprocessing_prompt = self.postprocessing_template.format_prompt(**info_dict)
        return self.llm.predict(postprocessing_prompt.to_string())

In [68]:
giga_key = os.environ.get("SB_AUTH_DATA")
giga_pro = GigaChat(credentials=giga_key, model="GigaChat", timeout=30, verify_ssl_certs=False)

traveler = MultiAgent(giga_pro, get_weather, get_url_booking_hotels, get_url_booking_tikets)
answer = traveler.run("Организуй поездку из Санкт-петербурга в Москву на 10 дней с 15.11.2024.")
print(answer)



> Entering new AgentExecutor chain...

Invoking: `get_weather` with `{'city': 'Москва'}`


city='Moscow' temperature=9.41 feels_like=8.0 weather='broken clouds' wind=2.68 message='Successfull'В Москве, куда вы направляетесь, температура будет около 9 градусов, с переменной облачностью и слабым ветром.

> Finished chain.


> Entering new AgentExecutor chain...

Invoking: `get_url_booking_hotels` with `{'city': 'Москва', 'date_in_str': '2024-11-15', 'date_out_str': '2024-11-25'}`


city='Москва' checkin_date='2024-11-15' checkout_date='2024-11-25' url='Booking link for hotels in Москва: https://travel.yandex.ru/hotels/search/?&adults=2&checkinDate=2024-11-15&checkoutDate=2024-11-25&childrenAges=0&geoId=213' message='Hotel booking search successful'Ваша ссылка для бронирования отеля в Москве с 15 по 25 ноября 2024 года: https://travel.yandex.ru/hotels/search/?&adults=2&checkinDate=2024-11-15&checkoutDate=2024-11-25&childrenAges=0&geoId=213.

> Finished chain.


> Entering new AgentExecu

In [71]:
print(answer)

- Погода в Москве с 15 по 25 ноября 2024 года:
          - Температура будет около 9 градусов.
          - Облачность переменная.
          - Слабый ветер.

        - Отели в Москве с 15 по 25 ноября 2024 года:
          - Ссылка для бронирования: https://travel.yandex.ru/hotels/search/?&adults=2&checkinDate=2024-11-15&checkoutDate=2024-11-25&childrenAges=0&geoId=213.

        - Авиабилеты Санкт-Петербург - Москва с 15 по 25 ноября 2024 года:
          - Ссылка для бронирования: https://travel.yandex.ru/avia/search/result/?&adults_seats=2&fromId=c2&klass=economy&oneway=2&return_date=2024-11-25&toId=c213&when=2024-11-15.
